In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(   #创建训练集
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(   #创建测试集
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)#将创建好的datasets作为dataloader的输入
test_dataloader = DataLoader(test_data, batch_size=64)  

In [4]:
#类似第6节分类神经网络
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()    
        self.flatten = nn.Flatten()              #Flatten将张量从第一个维度浓缩为一个维度
        self.linear_relu_stack = nn.Sequential( 
            nn.Linear(28*28, 512),    #线性层
            nn.ReLU(),                #非线性激活函数
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):           
        x = self.flatten(x)         
        logits = self.linear_relu_stack(x)   
        return logits

In [5]:
model = NeuralNetwork()  #实例化

In [6]:
#接下来开始训练模型，训练之前要定义一些超参数，还要设置一个目标函数loss函数
#超参数的值不需要优化，但会影响训练效果，比如batch size，学习率learning_rate，训练周期数epochs等
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [7]:
#交叉熵Loss函数
loss_fn = nn.CrossEntropyLoss() #直接把logits传入交叉熵函数中，在函数中会对logits归一化，然后计算预测误差

In [8]:
#接下来构建优化器  优化器中会设计一个算法进行对参数的更新
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer): #单步训练
    size = len(dataloader.dataset)        #计算dataloader的大小
    for batch, (X, y) in enumerate(dataloader):  #enumerate()，对dataloader枚举
        # Compute prediction and loss
        pred = model(X)  # 就是logits
        loss = loss_fn(pred, y)  #将logits和y放入loss_fn函数中得到loss标量
        # Backpropagation
        optimizer.zero_grad()  #梯度置零
        loss.backward()      #计算梯度
        optimizer.step()     #通过优化器对参数进行更新
        if batch % 100 == 0:  #每到100个就查看一下loss值
            loss, current = loss.item(), batch * len(X)  #查看当前训练了多少个样本
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test_loop(dataloader, model, loss_fn): #单步测试
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():  #不会计算梯度函数，效率更高
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss() #周期训练
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer) #循环调用两个单步步骤
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297170  [    0/60000]
loss: 2.287079  [ 6400/60000]
loss: 2.270108  [12800/60000]
loss: 2.264654  [19200/60000]
loss: 2.242713  [25600/60000]
loss: 2.205695  [32000/60000]
loss: 2.214495  [38400/60000]
loss: 2.179672  [44800/60000]
loss: 2.180261  [51200/60000]
loss: 2.131116  [57600/60000]
Test Error: 
 Accuracy: 44.7%, Avg loss: 2.134601 

Epoch 2
-------------------------------
loss: 2.148691  [    0/60000]
loss: 2.133478  [ 6400/60000]
loss: 2.076335  [12800/60000]
loss: 2.088089  [19200/60000]
loss: 2.029632  [25600/60000]
loss: 1.963990  [32000/60000]
loss: 1.982476  [38400/60000]
loss: 1.906371  [44800/60000]
loss: 1.918987  [51200/60000]
loss: 1.812640  [57600/60000]
Test Error: 
 Accuracy: 55.7%, Avg loss: 1.831597 

Epoch 3
-------------------------------
loss: 1.874301  [    0/60000]
loss: 1.834367  [ 6400/60000]
loss: 1.722998  [12800/60000]
loss: 1.757711  [19200/60000]
loss: 1.647437  [25600/60000]
loss: 1.598624  [32000/600